In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import Dataset
import bert_score
import os

from basic_trainer import get_basic_trainer
from KG_trainer_no_comp import get_KG_trainer
from evaluator import evaluate_model
import networkx as nx
import pickle

c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading cpnet...
Done
Loaded 3947 allowed concept IDs from train/val data.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
DATA_PATH = "data/eg"

In [4]:
trainer = get_KG_trainer(
        source_path=f"{DATA_PATH}/train.source",
        target_path=f"{DATA_PATH}/train.target",
        model_name="facebook/bart-base", 
        output_dir="KG_finetuned_out",
        max_len=128,
        epochs=1,
        train_batch_size=60
    )


Loading preprocessed dataset from disk...


Some weights of BartGraphAwareForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['graph_encoder.embedding.weight', 'graph_fusion_layer.bias', 'graph_fusion_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\quent\OneDrive\Desktop\Code\Efficient-Common-Sense-in-LLMs-via-Knowledge-Graph-Compression\KG_trainer.py:244: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
trainer.train()

Step,Training Loss
100,2.384300
200,2.150500
300,2.098300
400,2.071300


c:\Users\quent\anaconda3\envs\capstone_scratch_2\lib\site-packages\transformers\modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=427, training_loss=2.1687414093374926, metrics={'train_runtime': 1619.7091, 'train_samples_per_second': 15.803, 'train_steps_per_second': 0.264, 'total_flos': 305843942707200.0, 'train_loss': 2.1687414093374926, 'epoch': 1.0})

In [5]:
def generate_explanation(model, tokenizer, sentence: str) -> list:
    # Tokenize the input text
    inputs = tokenizer(sentence, return_tensors="pt").to(device)

    # Generate output using beam search:
    output_ids = model.generate(
        **inputs,
        max_length=60,
        num_beams=3,
        num_return_sequences=3,
        early_stopping=True
    )

    # Decode each of the generated token sequences
    explanations = [
        tokenizer.decode(ids, skip_special_tokens=True)
        for ids in output_ids
    ]
    return explanations

In [ ]:
from KG_trainer import get_graph_info, BartGraphAwareForConditionalGeneration

FINE_TUNED_MODEL_DIR = "KG_finetuned_out/checkpoint-427"

# Load the tokenizer and custom model.
tokenizer = BartTokenizer.from_pretrained(FINE_TUNED_MODEL_DIR)
model = BartGraphAwareForConditionalGeneration.from_pretrained(FINE_TUNED_MODEL_DIR)

# Move model to GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def generate_explanation_kg(sentence: str) -> list:
    # Tokenize the input sentence.
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    
    # Compute the graph features for the sentence.
    graph_info = get_graph_info(sentence)
    
    # Ensure the concept_ids tensor has a batch dimension.
    if graph_info["concept_ids"].dim() == 1:
        graph_info["concept_ids"] = graph_info["concept_ids"].unsqueeze(0)
    
    # Make sure graph features are on the same device.
    graph_info["concept_ids"] = graph_info["concept_ids"].to(device)
    
    # Generate outputs, passing the extra keyword argument "concept_ids".
    output_ids = model.generate(
        **inputs,
        concept_ids=graph_info["concept_ids"],
        max_length=60,
        num_beams=3,
        num_return_sequences=3,  # Added to return 3 distinct sequences
        early_stopping=True
    )
    
    explanations = [
        tokenizer.decode(ids, skip_special_tokens=True)
        for ids in output_ids
    ]
    return explanations

In [ ]:
input_sentence = "I can count the stars"
print("Generated explanation:", generate_explanation_kg(input_sentence))

Generated explanation: ['Stars cannot be counted.', 'Stars are too big to count.', 'Stars cannot be counted by humans.']


In [ ]:
input_sentence = "We use a book to tell the time"
print("Generated explanation:", generate_explanation_kg(input_sentence))

Generated explanation: ['A book is not used to tell time.', 'A book is not a tool to tell time.', 'A book is not used to tell the time.']


In [ ]:
input_sentence = "The 5 foot tall man and the 10 foot tall man ate breakfast together"
print("Generated explanation:", generate_explanation_kg(input_sentence))

Generated explanation: ['A 5 foot tall man cannot eat breakfast.', 'The 5 foot tall man and the 10 foot tall woman eat breakfast together.', 'The 5 foot tall man and the 10 foot tall woman cannot eat breakfast.']
